In [13]:
import requests
import json
import pandas as pd
from google.cloud import bigquery

re = requests.get("https://apac2-s3wrapper.capillarytech.com/core/v1/reward/brand/6", headers={"Authorization":"Basic YWRtaW46YWRtaW4="})
data = json.loads(re.content)
data = data['rewardList']

for i in data:
        for key in i.keys():
                i[key] = str(i[key])

df1 = pd.DataFrame.from_records(data)
data = []
for i in range(len(df1)):
    data.append(json.loads(df1.iloc[i, :].to_json()))

#find schema
schema = []
new_col_name = []
for name in data[0].keys():
    new_col_name.append("_".join(name.split(" ")))
    schema.append(bigquery.SchemaField("_".join(name.split(" ")), bigquery.enums.SqlTypeNames.STRING))

#connect to bigquery
client = bigquery.Client.from_service_account_json('google_token.json')
job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition="WRITE_TRUNCATE",
)
table_id = 'taptap-c2c5f.manual_dwh.capillary_reward_list'
print(table_id)

job = client.load_table_from_json(
    data, table_id, job_config=job_config
)
print(job.result())

taptap-c2c5f.manual_dwh.capillary_reward_list
LoadJob<project=taptap-c2c5f, location=US, id=995eb528-4ac2-46f9-a69a-9e8e68b605e5>
